In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve,roc_auc_score
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/classification')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT=0.3

    model = Sequential()
    model.add(Conv2D(32, (1, 5), padding='same', input_shape=(20, 1, 20),activation='relu', kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())

    model.add(Dense(16,activation='relu', kernel_constraint=maxnorm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(2, kernel_constraint=maxnorm(W_maxnorm), activation='softmax'))


    myoptimizer = RMSprop(learning_rate={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-06)
    mylossfunc = 'binary_crossentropy'
    model.compile(loss=mylossfunc, optimizer=myoptimizer, metrics=['accuracy'])
    # model.compile(loss=RMSprop, optimizer=myoptimizer,metrics=['accuracy'])
    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])

    # score, acc = model.evaluate(X_test,Y_test, verbose=False)
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK,'model':model}

In [ ]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  X_test_shuffled, Y_test_shuffled = shuffle(X_test, Y_test)
  return X_train_shuffled, Y_train_shuffled, X_test_shuffled, np.fliplr(Y_test_shuffled)

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32x1_16',
                                          verbose=False)

100%|██████████| 10/10 [01:59<00:00, 11.92s/it, best loss: 0.13187016546726227]


In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=(0.1))

Epoch 1/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2512 - accuracy: 0.9285 - val_loss: 0.2524 - val_accuracy: 0.9282
Epoch 2/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2484 - accuracy: 0.9292 - val_loss: 0.2478 - val_accuracy: 0.9297
Epoch 3/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2499 - accuracy: 0.9293 - val_loss: 0.2528 - val_accuracy: 0.9282
Epoch 4/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2441 - accuracy: 0.9306 - val_loss: 0.2548 - val_accuracy: 0.9260
Epoch 5/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2405 - accuracy: 0.9323 - val_loss: 0.2618 - val_accuracy: 0.9240
Epoch 6/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2460 - accuracy: 0.9309 - val_loss: 0.2520 - val_accuracy: 0.9290
Epoch 7/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2473 - accuracy: 0.9303 - val_loss: 0.2541 - val_accuracy: 0.9270
Epoch 

In [ ]:
y_pred = best_model.predict(X_test)
roc_auc_score(Y_test,y_pred)

0.9609816805920075

In [ ]:
best_model.save('./seq_32x1_16')

INFO:tensorflow:Assets written to: ./seq_32x1_16/assets
